import dependencies

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

import data using pd.read_csv()

In [15]:
start_year='1995'
end_year='2018'
GDP=pd.read_csv('API_NY.GDP.MKTP.CD_DS2_en_csv_v2_713242.csv', header=2)
CO2=pd.read_csv('API_EN.ATM.CO2E.PC_DS2_en_csv_v2_713061.csv', header=2)
pop=pd.read_csv('API_SP.POP.TOTL_DS2_en_csv_v2_713131.csv', header=2)
GDP.head()

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,Unnamed: 64
0,Aruba,ABW,GDP (current US$),NY.GDP.MKTP.CD,NaN,NaN,NaN,NaN,NaN,NaN,...,2.549721e+09,2.534637e+09,2.581564e+09,2.649721e+09,2.691620e+09,2.646927e+09,2.700559e+09,NaN,NaN,NaN
1,Afghanistan,AFG,GDP (current US$),NY.GDP.MKTP.CD,5.377778e+08,5.488889e+08,5.466667e+08,7.511112e+08,8.000000e+08,1.006667e+09,...,1.780428e+10,2.000162e+10,2.056105e+10,2.048487e+10,1.990711e+10,1.936264e+10,2.019176e+10,1.936297e+10,NaN,NaN
2,Angola,AGO,GDP (current US$),NY.GDP.MKTP.CD,NaN,NaN,NaN,NaN,NaN,NaN,...,1.117897e+11,1.280529e+11,1.367099e+11,1.457122e+11,1.161936e+11,1.011239e+11,1.221238e+11,1.057510e+11,NaN,NaN
3,Albania,ALB,GDP (current US$),NY.GDP.MKTP.CD,NaN,NaN,NaN,NaN,NaN,NaN,...,1.289087e+10,1.231978e+10,1.277628e+10,1.322824e+10,1.138693e+10,1.186135e+10,1.302506e+10,1.510250e+10,NaN,NaN
4,Andorra,AND,GDP (current US$),NY.GDP.MKTP.CD,NaN,NaN,NaN,NaN,NaN,NaN,...,3.442063e+09,3.164615e+09,3.281585e+09,3.350736e+09,2.811489e+09,2.877312e+09,3.013387e+09,3.236544e+09,NaN,NaN


In [17]:
GDP_per_capita=(GDP)/(pop)


TypeError: unsupported operand type(s) for /: 'str' and 'str'

check if 'Country Code' is unique, then set_index to 'Country Code'

In [8]:
GDP_per_capita=(GDP_per_capita['Country Code'].value_counts()>1).sum()+(CO2['Country Code'].value_counts()>1).sum()+(pop['Country Code'].value_counts()>1).sum()+(GDP['Country Code'].value_counts()>1).sum()
GDP_per_capita=GDP_per_capita.set_index('Country Code')
GDP=GDP.set_index('Country Code')
CO2=CO2.set_index('Country Code')
pop=pop.set_index('Country Code')

NameError: name 'GDP_per_capita' is not defined

put GDP in the units of Billions and population in the units of Millions

In [4]:
# GDP.loc[:, start_year:end_year]=GDP.loc[:, start_year:end_year]/1000000000
# pop.loc[:, start_year:end_year]=pop.loc[:, start_year:end_year]/1000000

create new DF with only relevant columns

In [5]:
# GDP=pd.concat([GDP.loc[:, :'Indicator Name'], GDP.loc[:, start_year:end_year]], axis=1)
# CO2=pd.concat([CO2.loc[:, :'Indicator Name'], CO2.loc[:, start_year:end_year]], axis=1)
# pop=pd.concat([pop.loc[:, :'Indicator Name'], pop.loc[:, start_year:end_year]], axis=1)
# GDP_per_capita=pd.concat([GDP_per_capita.loc[:, :'Indicator Name'], GDP_per_capita.loc[:, start_year:end_year]], axis=1)

In [6]:
# Separate countries by income
begin_year='1995'
end_year='2014'
years_df_list=[]
years_list=[str(each_year) for each_year in range(int(begin_year), int(end_year)+1)]
idx_co2_concat_df=co2_concat_df.set_index('Country Code')
idx_gni_concat_df=gni_concat_df.set_index('Country Code')
for each_year in years_list: 
    each_year_df=pd.concat([idx_co2_concat_df[each_year], 
                            idx_gni_concat_df[each_year]], 
                           axis=1, 
                           sort=False, 
                           keys=['CO2', 'Class'])
    years_df_list.append(each_year_df.groupby('Class').mean())
co2_trend_by_class=pd.concat(years_df_list, axis=1, keys=years_list)
co2_trend_by_class.T.plot()

check if GDP_per_capita is the same thing as GDP divided by population; the code uses only the '2018' column from each of the data set; the resulting few Falses is due to NA

In [7]:
combined_GDP_pop=pd.concat([index_GDP['2018'], index_pop['2018'], index_GDP_per_capita['2018']], axis=1)
combined_GDP_pop.columns=['GDP', 'pop', 'GDP_per_capita']
combined_GDP_pop['calculated']=(round(combined_GDP_pop['GDP']/combined_GDP_pop['pop'], 2)==round(combined_GDP_pop['GDP_per_capita'], 2))
combined_GDP_pop['calculated'].value_counts()
# combined_GDP_pop['calculated']==combined_GDP_pop['GDP_per_capita'].value_counts()

NameError: name 'index_GDP' is not defined

generate line graphs on the same plot

In [ ]:
single_country='USA'
one_country_gdp=combined_gdp[combined_gdp['Country Code']==single_country].loc[:, start_year:end_year].T
one_country_co2=combined_co2[combined_co2['Country Code']==single_country].loc[:, start_year:end_year].T
one_country_pop=combined_pop[combined_pop['Country Code']==single_country].loc[:, start_year:end_year].T
combined=pd.concat([one_country_gdp, one_country_co2, one_country_pop], axis=1)
combined.columns=['GDP', 'CO2', 'Population']
np.log(combined).plot()
plt.show()

binning GDP_per_caipta for 2018 and creating a country_labels DF for the labels

In [ ]:
GDP_per_capita['category']=pd.cut(GDP_per_capita['2018'], 6)
country_labels=GDP_per_capita[['Country Name', 'category']]

creating line graphs for each the mean CO2/capita per category per year

In [ ]:
labeled_CO2=CO2.join(country_labels['category'])
labeled_CO2.groupby('category').mean().T.plot()